In [2]:
import pandas as pd

C:\Users\Prerna\AppData\Local\Temp\ipykernel_8384\4080736814.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [5]:
data = pd.read_csv("C:\\Users\\Admin\\Desktop\\tsp_4.csv")


In [6]:
import math
import random
import time as tm
from tqdm import tqdm
import pandas as pd


class SolveTSPUsingACO:
    class Edge:
        def __init__(self, a, b, weight, initial_pheromone):
            self.a = a
            self.b = b
            if weight == 0:
                weight = 1e-10
            self.weight = weight
            self.pheromone = initial_pheromone

    class Ant:
        def __init__(self, alpha, beta, num_nodes, edges):
            self.alpha = alpha
            self.beta = beta
            self.num_nodes = num_nodes
            self.edges = edges
            self.tour = None
            self.distance = 0.0

        def _select_node(self):
            roulette_wheel = 0.0
            unvisited_nodes = [node for node in range(self.num_nodes) if node not in self.tour]
            heuristic_total = 0.0
            for unvisited_node in unvisited_nodes:
                heuristic_total += self.edges[self.tour[-1]][unvisited_node].weight
            for unvisited_node in unvisited_nodes:
                roulette_wheel += pow(self.edges[self.tour[-1]][unvisited_node].pheromone, self.alpha) * \
                                  pow((heuristic_total / self.edges[self.tour[-1]][unvisited_node].weight), self.beta)
            random_value = random.uniform(0.0, roulette_wheel)
            wheel_position = 0.0
            for unvisited_node in unvisited_nodes:
                wheel_position += pow(self.edges[self.tour[-1]][unvisited_node].pheromone, self.alpha) * \
                                  pow((heuristic_total / self.edges[self.tour[-1]][unvisited_node].weight), self.beta)
                if wheel_position >= random_value:
                    return unvisited_node

        def find_tour(self):
            self.tour = [random.randint(0, self.num_nodes - 1)]
            while len(self.tour) < self.num_nodes:
                self.tour.append(self._select_node())
            return self.tour

        def get_distance(self):
            self.distance = 0.0
            for i in range(self.num_nodes):
                self.distance += self.edges[self.tour[i]][self.tour[(i + 1) % self.num_nodes]].weight
            return self.distance

    def __init__(self, mode='ACS', colony_size=10, elitist_weight=1.0, min_scaling_factor=0.001, alpha=1.0, beta=3.0,
                 rho=0.1, pheromone_deposit_weight=1.0, initial_pheromone=1.0, steps=100, df=None):
        self.mode = mode
        self.colony_size = colony_size
        self.elitist_weight = elitist_weight
        self.min_scaling_factor = min_scaling_factor
        self.rho = rho
        self.pheromone_deposit_weight = pheromone_deposit_weight
        self.steps = steps
        self.num_nodes = len(df)
        self.df = df
        self.nodes = [(df['XCOORD.'][i], df['YCOORD.'][i]) for i in range(self.num_nodes)]
        self.labels = df['CUST NO.'].tolist()
        self.edges = [[None] * self.num_nodes for _ in range(self.num_nodes)]
        for i in range(self.num_nodes):
            for j in range(i + 1, self.num_nodes):
                self.edges[i][j] = self.edges[j][i] = self.Edge(i, j, math.sqrt(
                    pow(self.nodes[i][0] - self.nodes[j][0], 2.0) + pow(self.nodes[i][1] - self.nodes[j][1], 2.0)),
                                                                initial_pheromone)
        self.ants = [self.Ant(alpha, beta, self.num_nodes, self.edges) for _ in range(self.colony_size)]
        self.global_best_tour = None
        self.global_best_distance = float("inf")

    def _add_pheromone(self, tour, distance, weight=1.0):
        pheromone_to_add = self.pheromone_deposit_weight / distance
        for i in range(self.num_nodes):
            self.edges[tour[i]][tour[(i + 1) % self.num_nodes]].pheromone += weight * pheromone_to_add

    def _acs(self):
        for step in range(self.steps):
            for ant in self.ants:
                self._add_pheromone(ant.find_tour(), ant.get_distance())
                if ant.distance < self.global_best_distance:
                    self.global_best_tour = ant.tour
                    self.global_best_distance = ant.distance
            for i in range(self.num_nodes):
                for j in range(i + 1, self.num_nodes):
                    self.edges[i][j].pheromone *= (1.0 - self.rho)

    def run(self):
        start = tm.time()
        if self.mode == 'ACS':
            self._acs()
        runtime = tm.time() - start
        return runtime, self.global_best_distance

    def get_complete_tour(self):
        if self.global_best_tour is None:
            return None
        tour_labels = [self.df['CUST NO.'][i] for i in self.global_best_tour]
        return tour_labels


class HillClimbing:
    def __init__(self, nodes, global_best_tour, global_best_distance):
        self.nodes = nodes
        self.global_best_tour = global_best_tour
        self.global_best_distance = global_best_distance
        self.num_nodes = len(nodes)

    def _swap(self, tour, i, j):
        new_tour = tour.copy()
        new_tour[i], new_tour[j] = new_tour[j], new_tour[i]
        return new_tour

    def run(self):
        improvement = True
        while improvement:
            improvement = False
            for i in range(self.num_nodes - 1):
                for j in range(i + 1, self.num_nodes):
                    new_tour = self._swap(self.global_best_tour, i, j)
                    new_distance = self.calculate_distance(new_tour)
                    if new_distance < self.global_best_distance:
                        self.global_best_distance = new_distance
                        self.global_best_tour = new_tour
                        improvement = True

    def calculate_distance(self, tour):
        distance = 0.0
        for i in range(self.num_nodes):
            distance += math.sqrt(
                pow(self.nodes[tour[i]][0] - self.nodes[tour[(i + 1) % self.num_nodes]][0], 2.0) +
                pow(self.nodes[tour[i]][1] - self.nodes[tour[(i + 1) % self.num_nodes]][1], 2.0)
            )
        return distance


if __name__ == '__main__':
    _colony_size = 5
    _steps = 50

    df = data

    f = open("C:\\Users\\Prerna\\Desktop\\out.csv", 'w')

    f.write(','.join(['Iteration', 'ACS_time', 'ACS_dist', 'ACS_tour', 'HC_time', 'HC_dist', 'HC_tour']))
    f.write('\n')

    for i in range(10):
        print('Iter: ', i+1)
        for j in tqdm(range(20)):
            acs = SolveTSPUsingACO(mode='ACS', colony_size=_colony_size, steps=_steps, df=df)
            acs_time, acs_dist = acs.run()
            acs_tour = acs.get_complete_tour()
            
            hc = HillClimbing(acs.nodes, acs.global_best_tour, acs.global_best_distance)
            hc_start = tm.time()
            hc.run()
            hc_time = tm.time() - hc_start
            hc_dist = hc.global_best_distance
            hc_tour = [df['CUST NO.'][i] for i in hc.global_best_tour]
            
            f.write(str(i+1) + "," + str(acs_time) + "," + str(acs_dist) + "," + str(acs_tour) + "," + 
                    str(hc_time) + "," + str(hc_dist) + "," + str(hc_tour) + "\n")

    f.close()


Iter:  1


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:08<00:00,  2.33it/s]


Iter:  2


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:08<00:00,  2.48it/s]


Iter:  3


...

100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:08<00:00,  2.49it/s]


Iter:  9


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:07<00:00,  2.78it/s]


Iter:  10


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:07<00:00,  2.63it/s]


In [7]:
result = pd.read_csv("C:\\Users\\Prerna\\Desktop\\out.csv")

In [8]:
result

Iteration  \
1  0.223213 225.218164 [21  7  17 10 18 20 19 13  11 9   12  6  2  16 3   15  999 8   1  5  14 4]  0.415306 214.344209 [7   17 4  10 20 18 19  13 11 9   12  6  2  16 3           15   
   0.234442 222.667891 [19  13 11 12 9  6  2  16  3  15  999 8  5  1  17  7   21  14  4  10 20 18] 0.180008 220.082111 [19  13 11 9  12 6  2   16 3  15  999 8  5  1  17           7   
   0.249180 222.313613 [4   21 7  17 10 20 18 19  13 11  9   12 6  2  16  3   15  999 8  1  5  14] 0.173415 214.344209 [21  7  17 4  10 20 18  19 13 11  9   12 6  2  16           3   
   0.248749 231.630934 [21  7  17 20 18 19 13 11  9  12  6   2  16 3  15  999 8   5   1  10 4  14] 0.251549 220.082111 [14  4  10 20 18 19 13  11 9  12  6   2  16 3  15         999   
   0.176691 226.999522 [17  10 20 18 19 13 11 9   12 6   2   16 3  15 999 8   1   7   21 5  14 4]  0.068210 226.999522 [17  10 20 18 19 13 11  9  12 6   2   16 3  15 999          8   
...                                                                                                                                                                              ...   
10 0.201372 223.218409 [21  7  17 1  5  8  15 999 3  16  2   6  12 9  11  13  19  18  20 10 4  14] 0.140839 220.082111 [21  7  17 1  5  8  999 15 3  16  2   6  12 9  11          13   
   0.200475 229.071255 [999 15 3  16 2  6  12 9   11 13  19  18 20 10 17  1   5   21  7  4  14 8]  0.097610 229.071255 [999 15 3  16 2  6  12  9  11 13  19  18 20 10 17           1   
   0.245372 222.313613 [999 15 3  16 2  6  12 9   11 13  19  18 20 10 17  7   21  4   14 5  1  8]  0.337512 214.344209 [999 15 3  16 2  6  12  9  11 13  19  18 20 10 4           17   
   0.204247 227.959241 [13  11 12 9  6  2  16 3   15 999 8   1  5  14 21  7   17  10  18 20 19 4]  0.222185 214.344209 [13  11 9  12 6  2  16  3  15 999 8   1  5  14 21           7   
   0.222575 228.879171 [21  7  17 10 20 18 19 13  11 9   12  6  2  16 3   15  999 8   14 5  1  4]  0.226730 214.344209 [7   17 4  10 20 18 19  13 11 9   12  6  2  16 3           15   

                                                                                                                                                                           ACS_time  \
1  0.223213 225.218164 [21  7  17 10 18 20 19 13  11 9   12  6  2  16 3   15  999 8   1  5  14 4]  0.415306 214.344209 [7   17 4  10 20 18 19  13 11 9   12  6  2  16 3         999   
   0.234442 222.667891 [19  13 11 12 9  6  2  16  3  15  999 8  5  1  17  7   21  14  4  10 20 18] 0.180008 220.082111 [19  13 11 9  12 6  2   16 3  15  999 8  5  1  17         21   
   0.249180 222.313613 [4   21 7  17 10 20 18 19  13 11  9   12 6  2  16  3   15  999 8  1  5  14] 0.173415 214.344209 [21  7  17 4  10 20 18  19 13 11  9   12 6  2  16         15   
   0.248749 231.630934 [21  7  17 20 18 19 13 11  9  12  6   2  16 3  15  999 8   5   1  10 4  14] 0.251549 220.082111 [14  4  10 20 18 19 13  11 9  12  6   2  16 3  15          8   
   0.176691 226.999522 [17  10 20 18 19 13 11 9   12 6   2   16 3  15 999 8   1   7   21 5  14 4]  0.068210 226.999522 [17  10 20 18 19 13 11  9  12 6   2   16 3  15 999         1   
...                                                                                                                                                                             ...   
10 0.201372 223.218409 [21  7  17 1  5  8  15 999 3  16  2   6  12 9  11  13  19  18  20 10 4  14] 0.140839 220.082111 [21  7  17 1  5  8  999 15 3  16  2   6  12 9  11         19   
   0.200475 229.071255 [999 15 3  16 2  6  12 9   11 13  19  18 20 10 17  1   5   21  7  4  14 8]  0.097610 229.071255 [999 15 3  16 2  6  12  9  11 13  19  18 20 10 17          5   
   0.245372 222.313613 [999 15 3  16 2  6  12 9   11 13  19  18 20 10 17  7   21  4   14 5  1  8]  0.337512 214.344209 [999 15 3  16 2  6  12  9  11 13  19  18 20 10 4           7   
   0.204247 227.959241 [13  11 12 9  6  2  16 3   15 999 8   1  5  14 21  7   17  10  18 20 19 4]  0.222185 214.344209 [13  11 9  12